In [1]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
import warnings

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor


In [2]:
cancer = pd.read_csv('BreastCancer.csv',index_col=0)


In [3]:
X = cancer.drop('Class',axis=1)
y = cancer['Class']

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [5]:
gbc = GradientBoostingClassifier(random_state=24)
gbc.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 24,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [6]:
params = {'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
          'learning_rate': np.linspace(0.001,1,12),
          'max_features': [2,3,4,5,6]
         }

kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state =24)

In [7]:
gcv = GridSearchCV(gbc,param_grid = params, cv = kfold, scoring = 'neg_log_loss',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=GradientBoostingClassifier(random_state=24), n_jobs=-1,
             param_grid={'learning_rate': array([0.001     , 0.09181818, 0.18263636, 0.27345455, 0.36427273,
       0.45509091, 0.54590909, 0.63672727, 0.72754545, 0.81836364,
       0.90918182, 1.        ]),
                         'max_depth': [2, 3, 4],
                         'max_features': [2, 3, 4, 5, 6],
                         'n_estimators': [10, 20, 30, 50, 100, 200]},
             scoring='neg_log_loss', verbose=3)

In [8]:
print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 0.45509090909090905, 'max_depth': 2, 'max_features': 2, 'n_estimators': 30}
-0.09720436348731233


In [9]:
gcv = GridSearchCV(gbc,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=GradientBoostingClassifier(random_state=24), n_jobs=-1,
             param_grid={'learning_rate': array([0.001     , 0.09181818, 0.18263636, 0.27345455, 0.36427273,
       0.45509091, 0.54590909, 0.63672727, 0.72754545, 0.81836364,
       0.90918182, 1.        ]),
                         'max_depth': [2, 3, 4],
                         'max_features': [2, 3, 4, 5, 6],
                         'n_estimators': [10, 20, 30, 50, 100, 200]},
             scoring='roc_auc', verbose=3)

## XGBoost

In [11]:
print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 0.8183636363636363, 'max_depth': 3, 'max_features': 2, 'n_estimators': 30}
0.9941233289780318


In [12]:
!pip install xgboost

In [13]:
from xgboost import XGBClassifier

In [14]:
xgbm = XGBClassifier(random_state = 24)

In [15]:
le  = LabelEncoder()
y = le.fit_transform(y)
le.classes_


array(['Benign', 'Malignant'], dtype=object)

In [16]:
dict(zip(list(np.unique(y)),le.classes_))

{0: 'Benign', 1: 'Malignant'}

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [18]:
params = {'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
          'learning_rate': np.linspace(0.001,1,12),
          'max_features': [2,3,4,5,6],
          'booster':['gbtree', 'gblinear'],
          'num_parallel_tree':[2,3,4,5]
         }

kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state =24)

In [19]:
gcv = GridSearchCV(xgbm,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:07:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth", "max_features", "num_parallel_tree" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_typ...
                                     random_state=24, ...),
             n_jobs=-1,
             param_grid={'booster': ['gbtree', 'gblinear'],
                         'learning_rate': array([0.001     , 0.09181818, 0.18263636, 0.27345455, 0.36427273,
       0.45509091, 0.54590909, 0.63672727, 0.72754545, 0.81836364,
       0.90918182, 1.        ]),
                         'max_depth': [2, 3, 4],
                         'max_features': [2, 3, 4, 5, 6],
                         'n_estimators': [10, 20, 30, 50, 100, 200],
                         'num_parallel_tree': [2, 3, 4, 5]},
             scoring='roc_auc', verbose=3)

In [20]:
print(gcv.best_params_)
print(gcv.best_score_)

{'booster': 'gblinear', 'learning_rate': 0.18263636363636362, 'max_depth': 2, 'max_features': 2, 'n_estimators': 200, 'num_parallel_tree': 2}
0.994759062846352


## LightGBM Boosting

In [22]:
!pip install lightgbm

In [23]:
import lightgbm as lgb
print(lgb.__version__)


4.5.0


In [24]:
from lightgbm import LGBMClassifier

In [25]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [26]:
lgbm = LGBMClassifier(random_state = 24)
params = {'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
          'learning_rate': np.linspace(0.001,1,12),
          'max_features': [2,3,4,5,6],
          #'booster':['gbtree', 'gblinear'],
          #'num_parallel_tree':[2,3,4,5]
         }

kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state =24)

In [27]:
gcv = GridSearchCV(lgbm,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Info] Number of positive: 241, number of negative: 458
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 97
[LightGBM] [Info] Number of data points in the train set: 699, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.344778 -> initscore=-0.642072
[LightGBM] [Info] Start training from score -0.642072
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=LGBMClassifier(random_state=24), n_jobs=-1,
             param_grid={'learning_rate': array([0.001     , 0.09181818, 0.18263636, 0.27345455, 0.36427273,
       0.45509091, 0.54590909, 0.63672727, 0.72754545, 0.81836364,
       0.90918182, 1.        ]),
                         'max_depth': [2, 3, 4],
                         'max_features': [2, 3, 4, 5, 6],
                         'n_estimators': [10, 20, 30, 50, 100, 200]},
             scoring='roc_auc', verbose=3)

In [28]:
print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 1.0, 'max_depth': 3, 'max_features': 2, 'n_estimators': 30}
0.9925600621280524


## CatBoost

In [30]:
!pip install catboost

In [31]:
from catboost import CatBoostClassifier

In [32]:
cat = CatBoostClassifier(random_state=24,thread_count = -1)

params = {'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
          'learning_rate': np.linspace(0.001,1,12),
          'subsample': np.linspace(0,1,10)
          
          
         }

kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state =24)

In [33]:
gcv = GridSearchCV(cat,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 2160 candidates, totalling 10800 fits


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1080 fits failed out of a total of 10800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\catboost\core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline, use_best_model,

0:	learn: 0.3066444	total: 143ms	remaining: 1.29s
1:	learn: 0.1941173	total: 144ms	remaining: 576ms
2:	learn: 0.1402815	total: 145ms	remaining: 337ms
3:	learn: 0.1134874	total: 145ms	remaining: 218ms
4:	learn: 0.1003024	total: 146ms	remaining: 146ms
5:	learn: 0.0928531	total: 146ms	remaining: 97.6ms
6:	learn: 0.0857349	total: 147ms	remaining: 63ms
7:	learn: 0.0831798	total: 148ms	remaining: 36.9ms
8:	learn: 0.0790487	total: 149ms	remaining: 16.5ms
9:	learn: 0.0774218	total: 149ms	remaining: 0us


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x000001BF9C3D7920>,
             n_jobs=-1,
             param_grid={'learning_rate': array([0.001     , 0.09181818, 0.18263636, 0.27345455, 0.36427273,
       0.45509091, 0.54590909, 0.63672727, 0.72754545, 0.81836364,
       0.90918182, 1.        ]),
                         'max_depth': [2, 3, 4],
                         'n_estimators': [10, 20, 30, 50, 100, 200],
                         'subsample': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])},
             scoring='roc_auc', verbose=3)

In [34]:
print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 0.7275454545454545, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.7777777777777777}
0.9941642005583887


In [72]:
model = gcv.best_estimator_

In [74]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
roc_auc_score(y_test,y_pred)

0:	learn: 0.3141418	total: 636us	remaining: 5.73ms
1:	learn: 0.1966408	total: 1.5ms	remaining: 6.01ms
2:	learn: 0.1515722	total: 2ms	remaining: 4.68ms
3:	learn: 0.1237665	total: 2.46ms	remaining: 3.69ms
4:	learn: 0.1104060	total: 2.96ms	remaining: 2.96ms
5:	learn: 0.0996526	total: 3.44ms	remaining: 2.29ms
6:	learn: 0.0908386	total: 3.91ms	remaining: 1.68ms
7:	learn: 0.0879249	total: 4.36ms	remaining: 1.09ms
8:	learn: 0.0842406	total: 4.81ms	remaining: 534us
9:	learn: 0.0751872	total: 5.3ms	remaining: 0us


0.9716183574879227